In [2]:
#IMPORT some LIBS
import pandas as pd
import seaborn as sb
import numpy as np
import sys

#IMPORT DATASET
df = pd.read_csv('~/Documents/REU/final_w-bad-r.csv')
df.head()

,PDB_Code,Binding_Type,Binding_Value,Unit,Ligand_Name,Binding_Value_nM,Affinity_Category,SMILES,Sequence,Sequence_Length
0,4tmn,Ki,0.068,nM,0PK,0.068,high,CC(C)C[C@H](N[P@@](=O)([O-])[C@H](Cc1ccccc1)N[...,ITGTSTVGVGRGVLGDQKNINTTYSTYYYLQDNTRGDGIFTYDAKY...,316
1,5tmn,Ki,9.100,nM,0PJ,9.100,high,CC(C)C[C@H](NC(=O)[C@H](CC(C)C)N[P@](=O)([O-])...,ITGTSTVGVGRGVLGDQKNINTTYSTYYYLQDNTRGDGIFTYDAKY...,316
2,1ydr,Ki,3.000,uM,IQP,3000.000,low,C[C@H]1C[NH2+]CCN1S(=O)(=O)c1cccc2cnccc12,VKEFLAKAKEDFLKKWENPAQNTAHLDQFERIKTLGTGSFGRVMLV...,354
3,1ydt,Ki,48.000,nM,IQB,48.000,medium,O=S(=O)(NCC[NH2+]C/C=C/c1ccc(Br)cc1)c1cccc2cnc...,VKEFLAKAKEDFLKKWENPAQNTAHLDQFERIKTLGTGSFGRVMLV...,354
4,1bcu,Kd,0.530,mM,PRL,530000.000,low,Nc1ccc2cc3ccc(N)cc3[nH+]c2c1,AGLRPLFEKKSLEDIVEGSDAEIGMSPWQVMLFRKPQELLCGASLI...,249


In [3]:
#ONE HOT ENCODING SMILES
def smiles_to_onehot(smiles_list, max_length=None, char_to_idx=None):
    """
    Convert SMILES strings to one-hot encoded vectors.
    
    Parameters:
    smiles_list: List of SMILES strings
    max_length: Maximum length to pad/truncate sequences (if None, uses max length in data)
    char_to_idx: Dictionary mapping characters to indices (if None, creates from data)
    
    Returns:
    onehot_encoded: numpy array of shape (n_samples, max_length, n_unique_chars)
    char_to_idx: character to index mapping
    """
    
    # Get all unique characters if char_to_idx not provided
    if char_to_idx is None:
        all_chars = set()
        for smiles in smiles_list:
            all_chars.update(smiles)
        
        # Create character to index mapping
        unique_chars = sorted(list(all_chars))
        char_to_idx = {char: idx for idx, char in enumerate(unique_chars)}
    
    # Add padding character if not present
    if '<PAD>' not in char_to_idx:
        char_to_idx['<PAD>'] = len(char_to_idx)
    
    # Determine max length
    if max_length is None:
        max_length = max(len(smiles) for smiles in smiles_list)
    
    n_chars = len(char_to_idx)
    n_samples = len(smiles_list)
    
    # Initialize one-hot array
    onehot_encoded = np.zeros((n_samples, max_length, n_chars))
    
    # Fill the one-hot array
    for i, smiles in enumerate(smiles_list):
        for j, char in enumerate(smiles[:max_length]):  # Truncate if longer
            if char in char_to_idx:
                onehot_encoded[i, j, char_to_idx[char]] = 1
        
        # Pad with padding character if shorter
        for j in range(len(smiles), max_length):
            onehot_encoded[i, j, char_to_idx['<PAD>']] = 1
    
    return onehot_encoded, char_to_idx

def flatten_onehot_for_rf(onehot_encoded):
    """
    Flatten one-hot encoded SMILES for use with Random Forest.
    
    Parameters:
    onehot_encoded: numpy array of shape (n_samples, max_length, n_unique_chars)
    
    Returns:
    flattened: numpy array of shape (n_samples, max_length * n_unique_chars)
    """
    n_samples, max_length, n_chars = onehot_encoded.shape
    return onehot_encoded.reshape(n_samples, max_length * n_chars)

def encode_smiles_column(df, smiles_column, method='onehot'):
    """
    Encode SMILES column in DataFrame.
    
    Parameters:
    df: pandas DataFrame
    smiles_column: name of column containing SMILES
    method: 'onehot' or 'fingerprint'
    
    Returns:
    encoded_features: numpy array of encoded features
    """
    
    smiles_list = df[smiles_column].dropna().tolist()
    
    if method == 'onehot':
        # CORRECT way to use one-hot encoding
        onehot_encoded, char_to_idx = smiles_to_onehot(smiles_list)
        encoded_features = flatten_onehot_for_rf(onehot_encoded)
        print(f"One-hot encoded shape: {encoded_features.shape}")
        return encoded_features
    
    elif method == 'fingerprint':
        encoded_features = molecular_fingerprint_alternative(smiles_list)
        print(f"Fingerprint features shape: {encoded_features.shape}")
        return encoded_features
    
    else:
        raise ValueError("Method must be 'onehot' or 'fingerprint'")


def add_encoded_smiles_to_df(df, smiles_column, method='onehot', column_name='encoded_smiles'):
    """
    Add encoded SMILES features as a new column to the DataFrame.
    
    Parameters:
    df: pandas DataFrame
    smiles_column: name of column containing SMILES
    method: 'onehot' or 'fingerprint'
    column_name: name for the new column containing encoded features
    
    Returns:
    df: DataFrame with new encoded column added
    """
    
    # Get the encoded features
    encoded_features = encode_smiles_column(df, smiles_column, method)
    
    # Add as new column - each row contains the full encoded vector
    df[column_name] = [row for row in encoded_features]
    
    print(f"Added column '{column_name}' with shape: {encoded_features.shape}")
    print(f"Each row contains a vector of length: {encoded_features.shape[1]}")
    
    return df


df = add_encoded_smiles_to_df(df, 'SMILES', method='onehot', column_name='onehot_encoded')
df.head()

#np.set_printoptions(threshold=sys.maxsize)
#np.vstack(df.loc[2, 'onehot_encoded'])

One-hot encoded shape: (282, 5712)
Added column 'onehot_encoded' with shape: (282, 5712)
Each row contains a vector of length: 5712


,PDB_Code,Binding_Type,Binding_Value,Unit,Ligand_Name,Binding_Value_nM,Affinity_Category,SMILES,Sequence,Sequence_Length,onehot_encoded
0,4tmn,Ki,0.068,nM,0PK,0.068,high,CC(C)C[C@H](N[P@@](=O)([O-])[C@H](Cc1ccccc1)N[...,ITGTSTVGVGRGVLGDQKNINTTYSTYYYLQDNTRGDGIFTYDAKY...,316,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,5tmn,Ki,9.100,nM,0PJ,9.100,high,CC(C)C[C@H](NC(=O)[C@H](CC(C)C)N[P@](=O)([O-])...,ITGTSTVGVGRGVLGDQKNINTTYSTYYYLQDNTRGDGIFTYDAKY...,316,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1ydr,Ki,3.000,uM,IQP,3000.000,low,C[C@H]1C[NH2+]CCN1S(=O)(=O)c1cccc2cnccc12,VKEFLAKAKEDFLKKWENPAQNTAHLDQFERIKTLGTGSFGRVMLV...,354,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1ydt,Ki,48.000,nM,IQB,48.000,medium,O=S(=O)(NCC[NH2+]C/C=C/c1ccc(Br)cc1)c1cccc2cnc...,VKEFLAKAKEDFLKKWENPAQNTAHLDQFERIKTLGTGSFGRVMLV...,354,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1bcu,Kd,0.530,mM,PRL,530000.000,low,Nc1ccc2cc3ccc(N)cc3[nH+]c2c1,AGLRPLFEKKSLEDIVEGSDAEIGMSPWQVMLFRKPQELLCGASLI...,249,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [4]:
def add_onehot_protein_column(df, sequence_col='Sequence', onehot_col='onehot_sequence', amino_acids=None, max_length=None, padding_char='-'):
    """
    Add a single column with one-hot encoded protein sequences to a DataFrame.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame containing protein sequences
    sequence_col : str, default 'sequence'
        Name of the column containing protein sequences
    onehot_col : str, default 'onehot_sequence'
        Name of the new column to store one-hot encoded sequences
    amino_acids : list, optional
        List of amino acids to use for encoding. If None, uses standard + non-standard set
    max_length : int, optional
        Maximum sequence length. If None, uses the longest sequence in the data
    padding_char : str, default '-'
        Character to use for padding shorter sequences
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame with new one-hot encoded column added
    """
    
    if amino_acids is None:
        amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y',  # Standard 20
                      'U', 'O', 'B', 'Z', 'J', 'X',  # Non-standard: U=Sec, O=Pyl, B=Asx, Z=Glx, J=Xle, X=Unknown
                      padding_char]  # Padding character
    
    # Make a copy to avoid modifying the original DataFrame
    df_copy = df.copy()
    
    # Get sequences and find max length
    sequences = df_copy[sequence_col].astype(str).str.upper()
    
    if max_length is None:
        max_length = sequences.str.len().max()
    
    # Pad sequences to same length
    padded_sequences = sequences.str.ljust(max_length, padding_char)
    
    # Create amino acid to index mapping
    aa_to_idx = {aa: i for i, aa in enumerate(amino_acids)}
    
    # Create one-hot encoding for all sequences
    onehot_vectors = []
    
    for seq_idx, sequence in enumerate(padded_sequences):
        # Initialize one-hot matrix for this sequence
        onehot_matrix = np.zeros((max_length, len(amino_acids)))
        
        # Fill the one-hot matrix
        for pos, aa in enumerate(sequence):
            if aa in aa_to_idx:
                onehot_matrix[pos, aa_to_idx[aa]] = 1
            else:
                print(f"Warning: Unknown amino acid '{aa}' at position {pos} in sequence {seq_idx}")
        
        # Flatten the matrix to create a single vector
        onehot_vector = onehot_matrix.flatten()
        onehot_vectors.append(onehot_vector)
    
    # Add the one-hot column to the DataFrame
    df_copy[onehot_col] = onehot_vectors
    
    return df_copy


def onehot_encode_sequences(sequences, amino_acids=None, max_length=None, padding_char='-'):
    """
    One-hot encode protein sequences and return as list of numpy arrays.
    
    Parameters:
    -----------
    sequences : list or pandas.Series
        Protein sequences to encode
    amino_acids : list, optional
        List of amino acids to use for encoding
    max_length : int, optional
        Maximum sequence length. If None, uses the longest sequence
    padding_char : str, default '-'
        Character to use for padding shorter sequences
    
    Returns:
    --------
    list
        List of numpy arrays, each containing a one-hot encoded sequence
    """

    # 20 common AA, + X as unknown + padding
    # "[SEP]" depend on model chain separator
    # "[CLS]" "[END]" depend on model, start (represents the entire sequence), end
    if amino_acids is None:
        amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y',
                      'U', 'O', 'B', 'Z', 'J', 'X', padding_char]
    
    # Convert to list if pandas Series
    if hasattr(sequences, 'tolist'):
        sequences = sequences.tolist()
    
    # Convert to uppercase and find max length
    sequences = [str(seq).upper() for seq in sequences]
    
    if max_length is None:
        max_length = max(len(seq) for seq in sequences)
    
    # Pad sequences
    padded_sequences = [seq.ljust(max_length, padding_char) for seq in sequences]
    
    # Create amino acid to index mapping
    aa_to_idx = {aa: i for i, aa in enumerate(amino_acids)}
    
    # Create one-hot encoding for each sequence
    onehot_vectors = []
    
    for sequence in padded_sequences:
        # Initialize one-hot matrix
        onehot_matrix = np.zeros((max_length, len(amino_acids)))
        
        # Fill the matrix
        for pos, aa in enumerate(sequence):
            if aa in aa_to_idx:
                onehot_matrix[pos, aa_to_idx[aa]] = 1
        
        # Flatten and add to list
        onehot_vectors.append(onehot_matrix.flatten())
    
    return onehot_vectors


df = add_onehot_protein_column(df, sequence_col='Sequence', onehot_col='onehot_sequence')
df.head()

,PDB_Code,Binding_Type,Binding_Value,Unit,Ligand_Name,Binding_Value_nM,Affinity_Category,SMILES,Sequence,Sequence_Length,onehot_encoded,onehot_sequence
0,4tmn,Ki,0.068,nM,0PK,0.068,high,CC(C)C[C@H](N[P@@](=O)([O-])[C@H](Cc1ccccc1)N[...,ITGTSTVGVGRGVLGDQKNINTTYSTYYYLQDNTRGDGIFTYDAKY...,316,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
1,5tmn,Ki,9.100,nM,0PJ,9.100,high,CC(C)C[C@H](NC(=O)[C@H](CC(C)C)N[P@](=O)([O-])...,ITGTSTVGVGRGVLGDQKNINTTYSTYYYLQDNTRGDGIFTYDAKY...,316,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
2,1ydr,Ki,3.000,uM,IQP,3000.000,low,C[C@H]1C[NH2+]CCN1S(=O)(=O)c1cccc2cnccc12,VKEFLAKAKEDFLKKWENPAQNTAHLDQFERIKTLGTGSFGRVMLV...,354,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1ydt,Ki,48.000,nM,IQB,48.000,medium,O=S(=O)(NCC[NH2+]C/C=C/c1ccc(Br)cc1)c1cccc2cnc...,VKEFLAKAKEDFLKKWENPAQNTAHLDQFERIKTLGTGSFGRVMLV...,354,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1bcu,Kd,0.530,mM,PRL,530000.000,low,Nc1ccc2cc3ccc(N)cc3[nH+]c2c1,AGLRPLFEKKSLEDIVEGSDAEIGMSPWQVMLFRKPQELLCGASLI...,249,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [6]:
#TESTING ONE HOT ENCODING WAS DONE PROPERLY

#print(df["onehot_encoded"][8])
#print(len(df["onehot_sequence"][0]))
#print(max(df["Sequence_Length"]))
#for i in range(56997):
 #   if df["onehot_sequence"][0][i] == 1:
  #      print(f"{df['onehot_sequence'][0][i]} -- i is {i}")

#for i in range(56997):
#    if df["onehot_encoded"][0][i] == 1:
#        print(f"{df['onehot_encoded'][0][i]} -- i is {i}")

In [15]:
#DESIGNATING FEATURES(X) and PREDICTED VAR(y)
X = np.hstack([np.vstack(df['onehot_sequence'].values), 
               np.vstack(df['onehot_encoded'].values)])
y = df['Affinity_Category']

#SPLITTING TESTING AND TRAINING DATA 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [16]:
#Diagnosing issues with model, exmaining class balance in training splits and looking at sparcity
def diagnose_rf_issues(X_train, y_train, X_test, y_test):
    """
    Diagnose potential issues with Random Forest performance.
    """
    print("=== DATASET DIAGNOSTICS ===")
    print(f"Training set size: {X_train.shape}")
    print(f"Test set size: {X_test.shape}")
    print(f"Number of features: {X_train.shape[1]}")
    
    # Check class distribution
    from collections import Counter
    train_distribution = Counter(y_train)
    test_distribution = Counter(y_test)
    
    print(f"\nClass distribution in training set: {train_distribution}")
    print(f"Class distribution in test set: {test_distribution}")
    
    # Check for class imbalance
    class_counts = list(train_distribution.values())
    imbalance_ratio = max(class_counts) / min(class_counts)
    print(f"Class imbalance ratio: {imbalance_ratio:.2f}")
    
    if imbalance_ratio > 3:
        print("⚠️  SEVERE CLASS IMBALANCE detected!")
    
    # Check feature sparsity
    sparsity = np.mean(X_train == 0)
    print(f"Feature sparsity: {sparsity:.2%}")
    
    if sparsity > 0.95:
        print("⚠️  VERY SPARSE features detected!")
    
    # Check for identical features
    unique_features = np.unique(X_train, axis=1)
    print(f"Unique feature columns: {unique_features.shape[1]} out of {X_train.shape[1]}")
    
    return train_distribution, test_distribution


diagnose_rf_issues(X_train, y_train, X_test, y_test)

=== DATASET DIAGNOSTICS ===
Training set size: (225, 62709)
Test set size: (57, 62709)
Number of features: 62709

Class distribution in training set: Counter({'low': 99, 'medium': 68, 'high': 58})
Class distribution in test set: Counter({'medium': 23, 'low': 18, 'high': 16})
Class imbalance ratio: 1.71
Feature sparsity: 96.37%
⚠️  VERY SPARSE features detected!
Unique feature columns: 12930 out of 62709


(Counter({'low': 99, 'medium': 68, 'high': 58}),
 Counter({'medium': 23, 'low': 18, 'high': 16}))

In [17]:
#BUILDING RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [18]:
#SIMPLE MODEL SCORE, (IDK WHAT THIS IS MADE WITH)
y_pred = rf.predict(X_test)
rf.score(X_test,y_test)

0.43859649122807015

In [19]:
#ADVANCED METRICS
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        high       0.55      0.38      0.44        16
         low       0.48      0.67      0.56        18
      medium       0.33      0.30      0.32        23

    accuracy                           0.44        57
   macro avg       0.45      0.45      0.44        57
weighted avg       0.44      0.44      0.43        57



In [21]:
#HYPER PARAMATERS (We should do more hyperparameter optimization if possible)
rf2 = RandomForestClassifier(n_estimators = 200,
                             #criterion = 'entropy',
                             max_depth=20,
                             min_samples_split=5,
                             max_features='sqrt',
                             class_weight='balanced'
)
rf2.fit(X_train, y_train)
rf2.score(X_test, y_test)

0.543859649122807

In [22]:
y_pred2 = rf2.predict(X_test)
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

        high       0.67      0.62      0.65        16
         low       0.55      0.61      0.58        18
      medium       0.45      0.43      0.44        23

    accuracy                           0.54        57
   macro avg       0.56      0.56      0.56        57
weighted avg       0.54      0.54      0.54        57



In [28]:
#Using actual nM affinity value to make a REGRESSOR Random Forest
#DESIGNATING FEATURES(XR) and PREDICTED VAR(yR)
XR = np.hstack([np.vstack(df['onehot_sequence'].values), 
               np.vstack(df['onehot_encoded'].values)])
yR = df['Binding_Value_nM']

#SPLITTING TESTING AND TRAINING DATA
XR_train, XR_test, yR_train, yR_test = train_test_split(XR, yR, test_size=0.2)

In [29]:
#CREATING REGRESSOR MODEL - since the affinity values have not had the log function applied this model works very poorly
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr.fit(XR_train, yR_train)

RandomForestRegressor()

In [30]:
#METRICS
yR_pred = rfr.predict(XR_test)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
print(mean_absolute_error(yR_pred, yR_test))
print(mean_squared_error(yR_pred, yR_test))
print(r2_score(yR_test, yR_pred))

679794.6312208949
2675601530094.8213
-0.1015675225675714


In [31]:
#Hyperparam tuning
param_grid = {
    'n_estimators': [100, 200, 300], #number of decision trees
    'max_depth': [10,20,30], #depth of trees
    'min_samples_split':[2,5,10], #min number of samples at split node(WHAT THIS MEAN?)
    'min_samples_leaf': [1,2,3] #min number of samples at leaf node (WHAT THIS MEAN?)
} #all numbers of options multiplied together give CV value (WAT?)

In [32]:
#NEW MODEL FOR HYPERPARAMATIZATION
from sklearn.model_selection import GridSearchCV
rfr_cv = GridSearchCV(estimator=rfr, param_grid=param_grid, cv = 3, scoring='neg_mean_squared_error', n_jobs = -1)
rfr_cv.fit(XR_train, yR_train)

GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [10, 20, 30],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_squared_error')

In [128]:
#METRICS FOR NEW MODEL
yR_pred = rfr_cv.predict(XR_test)
print(mean_absolute_error(yR_pred, yR_test))
print(mean_squared_error(yR_pred, yR_test))
print(r2_score(yR_test, yR_pred))

299745.8107726913
441075063595.51746
0.0664456363990632
